In [1]:
from time import perf_counter
import copy

from numpy import fft, array
import random as rnd

RealNumber = RealField(240)
ComplexNumber = ComplexField(240)

def balance_mod(a,q):
    if not isinstance(a,(sage.modules.free_module_element.FreeModuleElement_generic_dense,sage.modules.vector_integer_dense.Vector_integer_dense,list)):
        a=a%q
        if a>q//2:
            a=-q+a
        return(a)
        
    for t in range(len(a)):
        a[t]=a[t]%q
        if a[t]>q//2:
            a[t]=-q+a[t]
    return(a)
    

def euclidnorm(v):
    if isinstance(v,sage.rings.complex_mpfr.ComplexNumber):
        return norm(v)
    if isinstance(v,(list,sage.modules.free_module_element.FreeModuleElement_generic_dense)):
        return(sum(euclidnorm(i)^2 for i in v))
    v_list = v.list()
    return sqrt(sum(i^2 for i in v_list))

def rfact(B):   #it works!!!
    F = B[0,0].parent()
    d = B.nrows()
    
    Q=matrix(F,d,d)
    Q[0]=B[0]
    
    i,j = 0,0
    #print(sum( (((B[j]*Q[i])/(Q[i]*Q[i]))*Q[i]) for i in range(j) ))
    for j in range(1,d):
        s = vector(F,d)
        for i in range(j):
            s+=((B[j]*Q[i])/(Q[i]*Q[i]))*Q[i]
        Q[j]=B[j]-s
    
    R=matrix(F,d,d)
    for j in range(d):
        for i in range(j+1):
            #R[i,j]=Q[i]*B[j]/(Q[i]*Q[i])
            R[i,j]=Q[i]*B[j]/(Q[i]*Q[i])
            
    return R.transpose(), Q

def invertibles(f):
    out=[]
    for i in range(f):
        if gcd(i,f)==1:
            out.append(i)
    return out

def minkowski_embedding(a, truncated=True):
    K = a.parent().fraction_field()
    sigmas = K.embeddings(ComplexField(240))
    
    if truncated:
        return [s(a) for s in sigmas[:len(sigmas)/2]]
    else:
        return [s(a) for s in sigmas]

def log_embedding(a,truncated=True):
    ac = minkowski_embedding(a,truncated)
    return vector(RR, [ln(abs(t)) for t in ac])

def random_rounding(v):
    for i in range(1,len(v)):
        #v[i]=ZZ( ceil(v[i]) if randrange(128)<64 else floor(v[i]) )
        v[i]=round(v[i])
        
#         fraction_part = v[i]-floor(v[i])
#         D = GeneralDiscreteDistribution([fraction_part/5,(1-fraction_part)/5,0.8])
#         d = D.get_random_element() in (0, 1, 2)
        
        v[i]=floor(v[i]) if d==0 else ceil(v[i]) if d==1 else round(v[i])
        
    v[0] = -sum([tmp for tmp in v[1:]])
    return vector(ZZ,v)

def round_matrix(M):
    M_ = matrix(ZZ,M.nrows(),M.ncols())
    for i in range(M.nrows()):
        for j in range(M.ncols()):
            M_[i,j] = ZZ(M[i,j].round())
    return M_

from fpylll import *
from fpylll.algorithms.bkz2 import BKZReduction as BKZ2

def round_babai(B_,t, prec = 10^50, wrt_lll=False):
    
    B = (prec*B_)
    B = round_matrix(B)
    #print('target',t*prec)
    
    Bint = IntegerMatrix(B.nrows(),B.ncols())
    U=IntegerMatrix.identity(B.ncols())
    
    for i in range(B.nrows()):
        for j in range(B.ncols()):
            Bint[i,j] = B[i,j]
            
    M = GSO.Mat(Bint,  float_type='mpfr')
    lll = LLL.Reduction(M)
    lll()
    
#     beta=8
#     flags = BKZ.AUTO_ABORT|BKZ.GH_BND|BKZ.VERBOSE|BKZ.MAX_LOOPS
#     #print(BKZ.DEFAULT_STRATEGY)
#     par = BKZ.Param(block_size=beta, flags=flags, max_loops=60)

#     bkz = BKZ2(M) 
    
#     print('b')
#     print(Bint)
    T = t*prec
    T = [ float(tmp) for tmp in T]
    
    v= vector(ZZ, M.babai(T) )
    if not wrt_lll:
        #print(B.ncols(), B.nrows(), len(v*matrix(ZZ,Bint)))
        tmp = B.solve_left( v*matrix(ZZ,Bint) )
        return tmp
    else:
        return v

def unit_reduce(a, prec=10^70, non_increase=False):
    K = a.parent().fraction_field()
    z_ = K.gen()
    f = z_.multiplicative_order()
    
    if f<=4:
        return(a)
    
    units = [z_^((1-(i))/2) * (1-z_^i)/(1-z_) for i in invertibles(f/2) ]
    #units = [(z_^i-1)/(z_-1) for i in invertibles(f/2) ]
    assert all( [tmp.is_unit() for tmp in units] )
    
    va = log_embedding(a)
    V = va.parent()
    d = V.dimension()
    One = vector([1]*d)
    
    t = ( va.dot_product( One ) / d) * One
    t= va-t   #t is va projected orthogonally to vector 1
    
    assert sum( [tmp for tmp in t] ) < 10^-10
    print(orbit_element)

    B=matrix([
        log_embedding(units[i]) for i in range(d)
    ])

    
    #print('y:',y)
#    y = vector([real(tmp) for tmp in list(y)])
#    y = y*B
    tmp = list( round_babai(B[1:,:],t,prec=prec) )

    y=tmp
    u =  prod(( units[i+1])^-y[i] for i in range(len(units)-1))
        

    #assert sum(y)<10^-6
    
    if not non_increase:
        return a*u
    else:
        tmp = a*u
        return a if euclidnorm(a)<euclidnorm(tmp) else tmp
    
def size_reduce(R):
    K=R[0,0].parent()
    d = R.nrows()
    U = matrix.identity(K,d)
    
    for i in range(d):
        #unit step is yet ignored
        
        ur = unit_reduce(R[i,i],non_increase=True)
        
        U[i]*=1/ur
        R[i]*=1/ur
        for j in range(i-1,-1,-1): 
            r = list( R[i,j] )   #note i <-> j coz trransposition!
            mu = [round(tmp) for tmp in r]
            mu = K(mu)
            U[i]+=(-mu)*U[j]   #also transposed
            R[i]+=(-mu)*R[j]
    return U

def ascend(K,v):
    
    qh = len(v)
    d_ = v[0].parent().degree()
    d=d_*qh
    z_=K.gen()
    
#     if isinstance(v,(list,sage.modules.free_module_element.FreeModuleElement_generic_dense)):
#         return vector( [ ascend(K,v[:qh]), ascend(K,v[qh:]) ] )
    
    out = sum( z_^i * v[i] for i in range(qh) )
    return(out)

# def descend(K,v):
#     L = v[0].parent()
#     d_ = L.degree()
#     d  = K.degree()
#     qh = d_/d
    
#     out=[]
#     for ranks in range(d/qh):
#         for i in range(qh):
#             v.append

def descend_rank2(K,v):
    L = v[0].parent()
    d_ = L.degree()
    d  = K.degree()
    qh=d_/d
    
    out = []
    for i in range(qh):
#         print(v[0].list()[i:d_:qh])
        out.append(K(v[0].list()[i:d_:qh]))
    for i in range(qh):
        out.append(K(v[1].list()[i:d_:qh]))
    return out

def GEuclide(L, Lptr,a,b):
    print('Currently at',Lptr)
        
    K = a.parent().fraction_field()
    print(K)
    print(L[Lptr])
    assert K.degree()==1 or K==L[Lptr]   #assert if the pointer to the field is correct
    
    if K==QQ or K.degree()<2:
        #a, b = ZZ(a), ZZ(b)
        #print('a,b',a,b)
        return xgcd(a,b)[1:]
    
    
    subK = L[Lptr-1]
    #print(GEuclide(L,Lptr-1,a.norm(subK),b.norm(subK)))
    a, b =K(a), K(b)
    mu, nu = GEuclide(L,Lptr-1,a.norm(subK),b.norm(subK))
    mu_, nu_ = mu * a^-1 * a.norm(subK), nu * b^-1 * b.norm(subK)
    W = matrix(K,[
        [a, b],
        [nu_,mu_]
    ])
    print('Upwards:',Lptr)
    #print(W)
    Q, R = rfact(W)
    print('rfact completed')
    V = size_reduce(R)
    
#     retrieving parasiting unit
    nu, mu = V[1]*W 
    unit = a*mu-b*nu
    
    print('GEuclide almost done!')
    
    assert unit.is_unit()
    
    return V[1]*W * (1/unit)
    return V[1]*W 

def Lift(L,K,v):
    
    a,b = ascend(K,v[0:2]), ascend(L,v[2:4])
    mu, nu = GEuclide(a,b)
    U = matrix(K,[
        [a,b],
        [nu,mu]
    ])
    


In [5]:
from fpylll import *

import numpy as np

def convolution(f,g):
    return (f * g) % (modulo)

def balancedmod(f,q):
    g = list(((f[i] + q//2) % q) - q//2 for i in list(range(N)))
    return ZZ[x](g)

def gen_invertable_f(x,d,N):
    Rq=Integers(q)[x].quotient_ring(modulo)
    baseq=Rq.base_ring()
    
    #generated poly in T(d+1,d)
    fq=Rq(0)
    stop_iter=d^2
    nump, numm = 0, 0 #КОЛИЧЕСТВО +1 И -1 В КОЭФФ
    i=0
    while i<=d:
        #print('debug +',i)
        coord=randrange(N)
        #если коэффициент свободен, то ставим его как 1
        if fq[coord]==baseq(0):
            nump+=1
            fq+=Rq(x^coord)
        else:
            #print('missed!')
            continue    
        stop_iter-=1
        i+=1
        if stop_iter==0:
            print('Error!')
            break
    i=0
    while i<d:
        #print('debug -',i)
        coord=randrange(N)
        #если коэффициент свободен, то ставим его как -1
        if fq[coord]==baseq(0):
            numm+=1
            fq-=Rq(x^coord)
        else:
            continue 
        i+=1
        stop_iter-=1
        if stop_iter==0:
            print('Error!')
            break
    print('+:', nump, '-:', numm)
    return balancedmod(fq,q)

def gen_g(x,d,N):
    Rq=Integers(q)[x].quotient_ring(modulo)
    baseq=Rq.base_ring()
    
    #generated poly in T(d+1,d)
    fq=Rq(0)
    stop_iter=d^2
    nump, numm = 0, 0 #КОЛИЧЕСТВО +1 И -1 В КОЭФФ
    i=0
    while i<d:
        #print('debug +',i)
        coord=randrange(N)
        #если коэффициент свободен, то ставим его как 1
        if fq[coord]==baseq(0):
            nump+=1
            fq+=Rq(x^coord)
        else:
            #print('missed!')
            continue    
        stop_iter-=1
        i+=1
        if stop_iter==0:
            print('Error!')
            break
    i=0
    while i<d:
        #print('debug -',i)
        coord=randrange(N)
        #если коэффициент свободен, то ставим его как -1
        if fq[coord]==baseq(0):
            numm+=1
            fq-=Rq(x^coord)
        else:
            continue 
        i+=1
        stop_iter-=1
        if stop_iter==0:
            print('Error!')
            break
    print('+:', nump, '-:', numm)
    return balancedmod(fq,q)


def invertmodprime(f,p):
    Zx.<x> = ZZ[]
    T = Zx.change_ring(Integers(p)).quotient(modulo)
    return Zx(lift(1 / T(f)))

def message_encoding(msg):
    m=0
    for i in range(len(msg)):
        m+=x^i*msg[i]
    return(m)

def Babai_CVP(Lattice, target, debug=False):
    
    then=time.perf_counter()
    M = Lattice.LLL()
    dt=time.perf_counter()-then
    if debug:
        print("LLL is done in", dt)
        
    then=time.perf_counter()   
    G = M.gram_schmidt()[0]
    dt=time.perf_counter()-then
    if debug:
        print("Gram Schmidt is done in")
    
    diff = target
    for i in reversed(range(M.nrows())):
        diff -=  M[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff

def Babai_CVP_BKZ(Lattice, target, block_size_):
    
    print("Reducing sublattice of dim",D,"with beta=",beta)
    then=time.perf_counter()

    M = Lattice.BKZ(block_size=block_size_)

    dt=time.perf_counter()-then
    print('Sublattice counted in',dt, 'sec')
    
    G = M.gram_schmidt()[0]
    diff = target
    for i in reversed(range(M.nrows())):
        diff -=  M[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff

def Babai_CVP_no_reduction(Lattice,  target):
    M = Lattice
    
    then=time.perf_counter()
    G = M.gram_schmidt()[0]
    dt=time.perf_counter()-then
    print('Gram Schmidt counted in',dt, 'sec')
    diff = target
    
    for i in reversed(range(M.nrows())):
        diff -=  M[i] * ((diff * G[i]) / (G[i] * G[i])).round()
    return target - diff

In [6]:
#NTRU keygen

#deg=80  - - - 200
#deg=96  - - - 288
#deg=108 - - - 324
#deg=120 - - - 225
#deg=144 - - - 432
#deg=192 - - - 576
#deg=384 - - - 1152
#modulo=cyclotomic_polynomial(200)

numfield_f = 64
modulo=cyclotomic_polynomial(numfield_f)

print("mod", modulo)
N=modulo.degree(x)

N, p, q, d = N, 3, 307, N//3
print(gcd(p,q),'=',gcd(N,q))
print(q,'>',(6*d+1)*p)
Zx.<x>=ZZ[x]

f= gen_invertable_f(x,d,N)
f=balancedmod(f,q)

g=gen_g(x,d,N)
g=balancedmod(g,q)

print("f=",f)
print("g=",g)

fp=invertmodprime(f,p)
fq=invertmodprime(f,q)

gp=invertmodprime(f,p)
gq=invertmodprime(f,q)

h = balancedmod(p * convolution(fq,g),q)
print(h)

#NTRU hack

ZpZ=Integers(p)
ZqZ=Integers(q)
Zxp=ZpZ[x]
Zxq=ZqZ[x]
Rp.<t>=Zxp.quotient_ring(modulo)
Rq.<t>=Zxq.quotient_ring(modulo)

print(Rp, "is constructed...")
print(Rq, "is constructed...")

hq=Rq(h)
print("h is",h)

mod x^32 + 1
1 = 1
307 > 183
+: 11 -: 10
+: 10 -: 10
f= x^31 + x^30 - x^29 + x^28 - x^27 - x^26 - x^25 + x^24 + x^23 + x^22 + x^21 + x^18 - x^17 - x^15 - x^14 + x^13 - x^9 + x^8 - x^6 - x^4 + x^3
g= x^30 + x^28 - x^26 - x^25 + x^24 - x^22 + x^21 - x^20 + x^19 + x^18 - x^12 - x^11 - x^10 - x^8 - x^6 + x^5 - x^4 + x^3 + x^2 + x
75*x^31 + 100*x^30 - 55*x^29 + 130*x^28 + 22*x^27 + 97*x^26 - 139*x^25 + 12*x^24 - 136*x^23 - 32*x^22 - 82*x^21 - 40*x^20 + 59*x^19 - 65*x^18 - 117*x^17 + 67*x^16 - 149*x^15 - 128*x^14 + 146*x^13 + 84*x^12 - 15*x^11 + 81*x^10 + 55*x^9 + 34*x^8 + 26*x^7 - 14*x^6 + 41*x^5 + 74*x^4 - 60*x^3 - 108*x^2 + 51*x + 18
Univariate Quotient Polynomial Ring in t over Ring of integers modulo 3 with modulus x^32 + 1 is constructed...
Univariate Quotient Polynomial Ring in t over Ring of integers modulo 307 with modulus x^32 + 1 is constructed...
h is 75*x^31 + 100*x^30 - 55*x^29 + 130*x^28 + 22*x^27 + 97*x^26 - 139*x^25 + 12*x^24 - 136*x^23 - 32*x^22 - 82*x^21 - 40*x^20 + 59*x^1

In [7]:
#preprocessing for babai CVp

roth=[hq*Rq(x^k) for k in range(N) ]
rotH=IntegerMatrix(N,N)

for i in range(N):
    for j in range(N):
        rotH[i,j]=Integer(roth[i][j])
        
Lh_full=IntegerMatrix(2*N,2*N)

for i in range(0,N):
    Lh_full[i,i]=1
    
for i in range(0,N):
    for j in range(N,2*N):
        Lh_full[i,j]=rotH[i][j-N]

for i in range(N,2*N):
    Lh_full[i,i]=q
print(Lh_full)

[ 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 100  75 ]
[ 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 100 ]
[ 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 ]
[ 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  55 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 ]
[ 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 177  55 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 ]
[ 0 0 0 0 0 1 0 0 0 

In [8]:
Rr=1

roth=[hq*Rq(x^k) for k in [Rr*i for i in range(N/Rr)] ]
print(roth)

rotH_=Matrix([[ZZ(roth[i][j]) for j in range(N) ] for i in range(N/Rr)])

rotH=IntegerMatrix(N/Rr,N)

for i in range(N/Rr):
    for j in range(N):
        rotH[i,j]=rotH_[i][j]

print()
print(rotH)

[75*t^31 + 100*t^30 + 252*t^29 + 130*t^28 + 22*t^27 + 97*t^26 + 168*t^25 + 12*t^24 + 171*t^23 + 275*t^22 + 225*t^21 + 267*t^20 + 59*t^19 + 242*t^18 + 190*t^17 + 67*t^16 + 158*t^15 + 179*t^14 + 146*t^13 + 84*t^12 + 292*t^11 + 81*t^10 + 55*t^9 + 34*t^8 + 26*t^7 + 293*t^6 + 41*t^5 + 74*t^4 + 247*t^3 + 199*t^2 + 51*t + 18, 100*t^31 + 252*t^30 + 130*t^29 + 22*t^28 + 97*t^27 + 168*t^26 + 12*t^25 + 171*t^24 + 275*t^23 + 225*t^22 + 267*t^21 + 59*t^20 + 242*t^19 + 190*t^18 + 67*t^17 + 158*t^16 + 179*t^15 + 146*t^14 + 84*t^13 + 292*t^12 + 81*t^11 + 55*t^10 + 34*t^9 + 26*t^8 + 293*t^7 + 41*t^6 + 74*t^5 + 247*t^4 + 199*t^3 + 51*t^2 + 18*t + 232, 252*t^31 + 130*t^30 + 22*t^29 + 97*t^28 + 168*t^27 + 12*t^26 + 171*t^25 + 275*t^24 + 225*t^23 + 267*t^22 + 59*t^21 + 242*t^20 + 190*t^19 + 67*t^18 + 158*t^17 + 179*t^16 + 146*t^15 + 84*t^14 + 292*t^13 + 81*t^12 + 55*t^11 + 34*t^10 + 26*t^9 + 293*t^8 + 41*t^7 + 74*t^6 + 247*t^5 + 199*t^4 + 51*t^3 + 18*t^2 + 232*t + 207, 130*t^31 + 22*t^30 + 97*t^29 + 168*t^

In [9]:
Lh=IntegerMatrix(N/Rr+N,N+N/Rr)
print((N/Rr+N,N+N/Rr))

for i in range(0,N/Rr):
    Lh[i,i]=1
    
for i in range(0,N/Rr):
    for j in range(N/Rr,N/Rr+N):
        Lh[i,j]=rotH[i][j-N/Rr]

for i in range(N/Rr,N/Rr+N):
    Lh[i,i]=q
print(Lh)

print(Lh.nrows,Lh.ncols)

(64, 64)
[ 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 100  75 ]
[ 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 100 ]
[ 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 252 ]
[ 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  55 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 130 ]
[ 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 177  55 207 232  18  51 199 247  74  41 293  26  34  55  81 292  84 146 179 158  67 190 242  59 267 225 275 171  12 168  97  22 ]
[ 0 0 0 0 0

In [10]:
from fpylll.algorithms.bkz2 import BKZReduction as BKZ2
FPLLL.set_precision(300)

#print(Lh_sub.nrows,Lh_sub.ncols)
print('Now well reduce', N,'x',N, 'matrix:')
import time
beta=18

#BKZ
print('Preperocessing...')
then=time.perf_counter()
GSO_Lh_sub = GSO.Mat(Lh, float_type='mpfr')
dt=time.perf_counter()-then


flags = BKZ.AUTO_ABORT|BKZ.GH_BND|BKZ.VERBOSE|BKZ.MAX_LOOPS
#print(BKZ.DEFAULT_STRATEGY)
par = BKZ.Param(block_size=beta, flags=flags, max_loops=20)

bkz = BKZ2(GSO_Lh_sub) 
print('Preprocessed in',dt, 'sec')

print("Reducing lattice of dim",N,"with beta=",beta)
then=time.perf_counter()

DONE = bkz(par)

dt=time.perf_counter()-then
print('Counted in',dt, 'sec')
B=Lh
print(B)

Now well reduce 32 x 32 matrix:
Preperocessing...
Preprocessed in 0.007358581999142189 sec
Reducing lattice of dim 32 with beta= 18
{"i":        0,  "cputime": 0.047806,  "walltime": 0.056375,  "preproc": 0.038383,  "svp": 0.003387,  "#enum": 3716.000,  "lll": 0.036228,  "pruner": 0.001266,  "r_0":  201.000,  "/": 0.026850}
{"i":        1,  "cputime": 0.014835,  "walltime": 0.014791,  "preproc": 0.009304,  "svp": 0.001050,  "#enum": 2838.000,  "lll": 0.007280,  "pruner": 0.001065,  "r_0":  201.000,  "/": 0.026850}
Counted in 1.4774067580001429 sec
[ -1  1 -1  1   1   0  0  0 -1  1  0  1  0 -1  1  0   0  0 -1  1  1  0  1 -1  0  0 -1 -1 -1 -1  1  1   0  3   0   3   0   0 -3 -3 -3  3  -3   3  0  3  0  3  3   3  0  0  0  0  0 -3 -3  3 -3  3  0 -3   3  3 ]
[  0  1 -1  0  -1   0  1 -1  0  0  0  1 -1 -1  0 -1   1  0  0  1  1  1  1 -1 -1 -1  1 -1  1  1  0  0   3  3  -3   3  -3   0 -3  0 -3 -3  -3   0  0  0  0  0  3   3 -3  3 -3  0  3 -3 -3  0  3  0  3  0   0 -3 ]
[  1  0  0  0  -1   1  1  0  1

In [11]:
K = CyclotomicField(numfield_f)

BK = matrix(K,[
    [K(1),h],
    [0,K(q)]
])

R, Q = rfact(BK)


ab = vector([K(list(B[0])[:N]) , K(list(B[0])[N:])])
ab = ab*Q^-1
ab

(-1893179132882614697733640786576965479514024430712282795377048128131321960530889617451070686168229046803119052426888841538818838928453368440128211699712149878/45929159220409801271899747623062105380955411926677864040707274545542597104755514549050821493317380232938411984972926690150292720553505622692574082840074596161*zeta64^31 - 1853363283961448824122568776255567036183799849884754376532589945131570743503613657775770008050909175442200941245064851180522419081775221964842785014859617309/45929159220409801271899747623062105380955411926677864040707274545542597104755514549050821493317380232938411984972926690150292720553505622692574082840074596161*zeta64^30 - 991552167911601322869851396569455853947240054107541454631602679774058897098726959641038183392704728949654643310395385810619547153596965696030522079025049536/45929159220409801271899747623062105380955411926677864040707274545542597104755514549050821493317380232938411984972926690150292720553505622692574082840074596161*zeta64^29 - 361145752676

In [12]:
abt = R.solve_left(ab)

a, b = abt[0], abt[1]
a, b

(zeta64^31 + zeta64^30 - zeta64^29 - zeta64^28 - zeta64^27 - zeta64^26 - zeta64^23 + zeta64^22 + zeta64^20 + zeta64^19 - zeta64^18 + zeta64^14 - zeta64^13 + zeta64^11 + zeta64^9 - zeta64^8 + zeta64^4 + zeta64^3 - zeta64^2 + zeta64 - 1,
 -zeta64^31 + zeta64^28 - zeta64^27 + 2*zeta64^26 - zeta64^25 - 2*zeta64^23 - zeta64^22 + 3*zeta64^18 - zeta64^13 + zeta64^12 + zeta64^11 + zeta64^9 - zeta64^7 + zeta64^6 - zeta64^5 - 2*zeta64^4 + zeta64^3 - 2*zeta64^2)

In [14]:
L = [CyclotomicField(i) for i in [2,4,8,16,32,64,128]]

mn = GEuclide(L,5,abt[0],abt[1])

Currently at 5
Cyclotomic Field of order 64 and degree 32
Cyclotomic Field of order 64 and degree 32
Currently at 4
Cyclotomic Field of order 32 and degree 16
Cyclotomic Field of order 32 and degree 16
Currently at 3
Cyclotomic Field of order 16 and degree 8
Cyclotomic Field of order 16 and degree 8
Currently at 2
Cyclotomic Field of order 8 and degree 4
Cyclotomic Field of order 8 and degree 4
Currently at 1
Cyclotomic Field of order 4 and degree 2
Cyclotomic Field of order 4 and degree 2
Currently at 0
Rational Field
Cyclotomic Field of order 2 and degree 1
Upwards: 1
rfact completed
GEuclide almost done!
Upwards: 2
rfact completed
GEuclide almost done!
Upwards: 3
rfact completed
GEuclide almost done!
Upwards: 4
rfact completed
GEuclide almost done!
Upwards: 5
rfact completed
GEuclide almost done!


In [15]:
U = matrix([
    [a,b],
    [mn[0],mn[1]]
])
print(U.det().is_unit())

U

True


[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [16]:
det(U)

1

In [17]:
R_ = R*U
M=R_*Q

In [18]:
print(M)

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [19]:
for i in range(2):
    for j in range(2):
        print([t for t in M[i,j]])

[-...]


In [20]:
v0 = vector(list(M[0][0])+list(M[0][1]))
v1 = vector(list(M[1][0])+list(M[1][1]))

print(v0)
print(v1)

(-...)


In [21]:
Lh = matrix(Lh)
w0 = Lh.solve_left(v0%q)   #what???
print(w0)
w1 = Lh.solve_left(v1%q)
print(w1)

(-22, -11, 0, -2, -18, -21, -7, 3, -17, -12, -13, -9, -10, 23, -18, 1, 22, -3, -16, 8, -15, -9, 11, -14, -6, -9, 3, -1, -12, -17, 7, -15, 1, -1, -6, 9, 12, -7, 7, -1, -3, -2, -2, -2, 2, 11, -3, 0, 9, 10, 5, 3, 14, -7, 2, -1, 3, -8, 10, -3, -6, -1, -3, -2)
(63, -18, 5, -20, -26, -51, -31, -19, -28, 0, -30, -14, -16, 19, -36, -23, 38, -24, -3, 29, -9, -24, 37, -36, 5, -20, -23, -24, -17, -36, 6, -32, -4, -4, -7, 0, 2, -1, 0, -5, 3, -4, 3, -4, -12, 2, -4, -4, -1, 7, -3, -3, 10, 2, -2, 9, -2, 1, 1, -5, -4, -6, 0, -5)


In [22]:
print('answer:')
v0 = w0*Lh
v1 = w1*Lh
print( v0 )
print( v1 )
print('- - -')
print( balance_mod(list(v0),q))
print( balance_mod(list(v1),q))

answer:
(306, 1, 306, 1, 1, 0, 0, 0, 306, 1, 0, 1, 0, 306, 1, 0, 0, 0, 306, 1, 1, 0, 1, 306, 0, 0, 306, 306, 306, 306, 1, 1, 0, 3, 0, 3, 0, 0, 304, 304, 304, 3, 304, 3, 0, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, 304, 304, 3, 304, 3, 0, 304, 3, 3)
(57, 104, 246, 272, 32, 39, 112, 132, 260, 153, 194, 178, 200, 256, 163, 296, 77, 86, 212, 212, 144, 248, 276, 27, 156, 53, 55, 12, 73, 232, 143, 157, 246, 53, 76, 161, 268, 125, 129, 25, 288, 133, 55, 265, 50, 85, 47, 236, 166, 247, 280, 223, 233, 9, 299, 15, 171, 288, 95, 34, 252, 253, 124, 29)
- - -
[-1, 1, -1, 1, 1, 0, 0, 0, -1, 1, 0, 1, 0, -1, 1, 0, 0, 0, -1, 1, 1, 0, 1, -1, 0, 0, -1, -1, -1, -1, 1, 1, 0, 3, 0, 3, 0, 0, -3, -3, -3, 3, -3, 3, 0, 3, 0, 3, 3, 3, 0, 0, 0, 0, 0, -3, -3, 3, -3, 3, 0, -3, 3, 3]
[57, 104, -61, -35, 32, 39, 112, 132, -47, 153, -113, -129, -107, -51, -144, -11, 77, 86, -95, -95, 144, -59, -31, 27, -151, 53, 55, 12, 73, -75, 143, -150, -61, 53, 76, -146, -39, 125, 129, 25, -19, 133, 55, -42, 50, 85, 47, -71, -141, -60, -27, -

In [23]:
# print( euclidnorm(w0*Lh).n() )
print('- - - bal mod:')
print( euclidnorm(balance_mod(list(w0*Lh),q)).n() )
print( euclidnorm(w1*Lh).n() )
#print( euclidnorm(balance_mod(list(w1*Lh),q)).n() )
print('- - -')
print( euclidnorm(balance_mod(list(w0*Lh),q)).n() / euclidnorm(list(B[0])) )
print( euclidnorm(w0*Lh).n() / euclidnorm(list(B[0])) )

- - - bal mod:
201.000000000000
1423.40682870359
- - -
1.00000000000000
6.44044599159708


In [63]:
Lh.solve_left(v0)

(23940, -15016, -3042, 4178, 10874, 3761, 5205, 4725, -11089, 862, 7135, 7186, 3796, -14152, 1836, 1118, -4443, 5065, 9800, 10305, 2486, 3044, -3050, 239, -1025, -4947, -9209, 5318, -2923, 4752, -8275, 7081, 4975, -2133, 7464, -6106, 3921, 2838, -1747, 5972, -3199, -4517, -2032, -268, 408, 10, 1886, -3015, -2866, 1035, 3167, 1203, -158, -1195, 1524, -2734, 1268, 569, -284, -493, 1536, 1395, -973, -359, -912, -1345, 1417, 963, 1526, -328, -506, 269, -345, 198, -1109, 131, -133, 711, 281, 370, 842, 153, -31, -416, -19, 306, -358, 362, -213, -170, -28, 367, -126, -383, 261, -153, -45, 150, 73, -273, 323, 218, -125, 122, -111, 231, 121, -195, 165, 35, 40, 91, -81, -98, -91, 79, 28, -8, 71, 41, 60, 16, 1, 4, 83, -24, -148, 99)